In [26]:
import pandas as pd
import numpy as np
import pyspark
import math
from pyspark import SparkContext, SparkConf
from pyspark.mllib.recommendation import ALS, Rating, MatrixFactorizationModel

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import (IntegerType, 
                               StringType, 
                               StructType, 
                               StructField)

from tqdm.notebook import tqdm
import pandas as pd

conf = SparkConf()


In [2]:
# Kubernetes is a Spark master in my setup. on my little cluster

conf.setAppName("Last Descriptive Analysis").setMaster("k8s://https://192.168.4.60:6443") 

# Worker pods are created from the base Spark docker image.
# If you use another image, specify its name instead.

conf.set(
    "spark.kubernetes.container.image", 
    "johnb340/spark-executor:v1alpha") 


conf.set(
    "spark.kubernetes.container.image.pullPolicy", 
    "IfNotPresent") 

# Authentication certificate and token (required to create worker pods):
conf.set(
    "spark.kubernetes.authenticate.caCertFile", 
    "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt")
conf.set(
    "spark.kubernetes.authenticate.oauthTokenFile", 
    "/var/run/secrets/kubernetes.io/serviceaccount/token")

# Service account which should be used for the driver
conf.set(
    "spark.kubernetes.authenticate.driver.serviceAccountName", 
    "spark-minion") 

# 2 pods/workers will be created. Can be expanded for larger workloads.
conf.set("spark.executor.instances", "5") 

# The DNS alias for the Spark driver. Required by executors to report status.
conf.set( "spark.driver.host", "jupyter-spark-driver.default")

conf.set("spark.pyspark.driver.python","jupyter")

conf.set("spark.pyspark.driver.python.opts","notebook")

conf.set("spark.kubernetes.namespace","default")

conf.set("spark.driver.port", "20020")

# Initialize spark context, create executors
sc = pyspark.SparkContext(conf=conf)

23/02/24 03:46:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
text_file = "data/balancedRDD.csv"

In [7]:
df = pd.read_csv(text_file, error_bad_lines=False)

/tmp/ipykernel_9952/2856737018.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(text_file, error_bad_lines=False)


In [8]:
spark = SparkSession(sc)

In [9]:
df =  spark.createDataFrame(df)

/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [10]:
RDD = df.rdd

In [11]:
RDD.take(5)

23/02/24 03:48:06 WARN TaskSetManager: Stage 0 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.


[Row(564500=568200,  840854315=842801245,  27185=27264,   vanessa hudgens='  emilie autumn',  216=91),
 Row(564500=569400,  840854315=845566037,  27185=27317,   vanessa hudgens='  absynthe minded',  216=87),
 Row(564500=569500,  840854315=846311499,  27185=27328,   vanessa hudgens='  r. kelly',  216=6),
 Row(564500=569700,  840854315=850268306,  27185=27333,   vanessa hudgens='  westlife',  216=156),
 Row(564500=571100,  840854315=855644453,  27185=27408,   vanessa hudgens='  morbid angel',  216=37)]

In [16]:
RDD.map(lambda x:(int(x[4]))).max()

23/02/24 03:50:36 WARN TaskSetManager: Stage 3 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.


699

In [17]:
#Im going to norm the ratings column by dividing by max value
modelRDD = RDD.map(lambda x : ((x[1]),(x[2]),(int(x[4]))/699))

In [18]:
modelRDD.take(5)

23/02/24 03:50:49 WARN TaskSetManager: Stage 4 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.


[(842801245, 27264, 0.1301859799713877),
 (845566037, 27317, 0.12446351931330472),
 (846311499, 27328, 0.008583690987124463),
 (850268306, 27333, 0.22317596566523606),
 (855644453, 27408, 0.05293276108726753)]

In [19]:
# for model we only want user, artist and rating
finalRDD = modelRDD.map(lambda x: (x[0],x[1],x[2]))

In [20]:
finalRDD.take(5)

23/02/24 03:50:53 WARN TaskSetManager: Stage 5 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.


[(842801245, 27264, 0.1301859799713877),
 (845566037, 27317, 0.12446351931330472),
 (846311499, 27328, 0.008583690987124463),
 (850268306, 27333, 0.22317596566523606),
 (855644453, 27408, 0.05293276108726753)]

In [21]:
finalRDD.map(lambda x: (x[2])).stats()

23/02/24 03:51:27 WARN TaskSetManager: Stage 6 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.


(count: 444041, mean: 0.19855469081028176, stdev: 0.20862379377754262, max: 1.0, min: 0.001430615164520744)

In [22]:
#Now that we have our data in the format we need, we can split the data relevant to our algorithmm (_RDD)
# Into the testing, traing, and validation sets to develop and tet our model
rdd_training, rdd_validation, rdd_test = finalRDD.randomSplit([6,2,2], seed=5)
predict_validation = rdd_validation.map(lambda x: (x[0], x[1]))
predict_test = rdd_test.map(lambda x: (x[0], x[1]))



In [23]:
# cach so performance is faster an more stable
rdd_training.cache()
rdd_validation.cache()

PythonRDD[17] at RDD at PythonRDD.scala:53

In [ ]:
ALS.checkpointInterval = 2
seed=5
ranks = [5, 10, 20, 40, 80]
lambdas_ = [0.01, .10, 1., 10.]
lambdas_.sort()
iter_array = [2, 5, 10, 15, 20]



best_rank = -1
best_iteration = -1

best_params = {}
best_params['n_factors'] = ranks[0]
best_params['lam'] = lambdas_[0]
best_params['n_iter'] = 0
best_params['test_mse'] = float('inf')
best_params['min_error'] = float('inf')
best_params['model'] = None

for rank in ranks:
    print('Rank: {:d}'.format(rank))
    
    for lam in lambdas_:
        errors = np.zeros(20)
        err = 0
        print('Regularization: {:f}'.format(lam))
        for _iter in iter_array:
            print('Iterations: {:d}'.format(_iter))
            model = ALS.train(rdd_training, iterations=_iter, rank=rank,lambda_=lam, seed=seed)
            predictions = model.predictAll(predict_validation).map(lambda r: ((r[0], r[1]), r[2]))
            rates_and_predicts = rdd_validation.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).\
            join(predictions)
  
            error = math.sqrt(rates_and_predicts.map(lambda rec: (rec[1][0] - rec[1][1])**2).mean())
            errors[err] = error
            err += 1
            if  error < best_params['test_mse']:
                best_params['n_factors'] = rank
                best_params['lam'] = lam
                best_params['n_iter'] = _iter
                best_params['test_mse'] = error
                best_params['model'] = model
                model = None
                predictions = sc.emptyRDD()
                rates_and_predicts = sc.emptyRDD()
print('New optimal hyperparameters')
print(pd.Series(best_params))



Rank: 5
Regularization: 0.010000
Iterations: 2


23/02/24 03:57:47 WARN TaskSetManager: Stage 7 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:57:48 WARN TaskSetManager: Stage 8 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:57:48 WARN TaskSetManager: Stage 9 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:57:49 WARN TaskSetManager: Stage 10 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:57:58 WARN TaskSetManager: Stage 61 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:58:00 WARN TaskSetManager: Stage 62 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:58:03 WARN TaskSetManager: Stage 70 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 

Iterations: 5


23/02/24 03:58:12 WARN TaskSetManager: Stage 77 contains a task of very large size (3001 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:58:19 WARN TaskSetManager: Stage 164 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:58:20 WARN TaskSetManager: Stage 165 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:58:24 WARN TaskSetManager: Stage 179 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:58:27 WARN TaskSetManager: Stage 181 contains a task of very large size (3001 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:58:30 WARN TaskSetManager: Stage 183 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 03:58:30 WARN TaskSetManager: Stage 184 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.


Iterations: 10


...

23/02/24 04:43:16 WARN TaskSetManager: Stage 15700 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:43:16 WARN TaskSetManager: Stage 15701 contains a task of very large size (2999 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:43:38 WARN TaskSetManager: Stage 15752 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:43:38 WARN TaskSetManager: Stage 15753 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:43:42 WARN TaskSetManager: Stage 15760 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:43:45 WARN TaskSetManager: Stage 15763 contains a task of very large size (2970 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:43:51 WARN TaskSetManager: Stage 15765 contains a task of very large size (2969 KiB). The maximum recommended task siz

Iterations: 5


23/02/24 04:43:51 WARN TaskSetManager: Stage 15767 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:43:51 WARN TaskSetManager: Stage 15768 contains a task of very large size (3002 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:44:38 WARN TaskSetManager: Stage 15855 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:44:39 WARN TaskSetManager: Stage 15856 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:44:43 WARN TaskSetManager: Stage 15869 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:44:46 WARN TaskSetManager: Stage 15872 contains a task of very large size (2999 KiB). The maximum recommended task size is 1000 KiB.
23/02/24 04:44:51 WARN TaskSetManager: Stage 15874 contains a task of very large size (2969 KiB). The maximum recommended task siz

Iterations: 10


23/02/24 04:44:51 WARN TaskSetManager: Stage 15877 contains a task of very large size (3001 KiB). The maximum recommended task size is 1000 KiB.


In [ ]:
#This model wants to stay at .12 rmse. We can see this below, with much lower params. It converges quickly

In [14]:
# Now for the tst dataset
model = ALS.train(rdd_training, rank=5, iterations=5, seed=5)
predictions = model.predictAll(predict_test).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = rdd_test.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())

print('The error for the test data is {:.2f}'.format(error))

The error for the test data is 0.12


In [17]:
rdd_training, rdd_test = modelRDD.randomSplit([7, 3], seed=5)

complete_model = ALS.train(rdd_training, rank=5, iterations=5,seed=5)

In [18]:
predict_test = rdd_test.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(predict_test).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = rdd_test.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())

print('For testing data the RMSE is {:.2f}'.format(error))

For testing data the RMSE is 0.12


In [20]:
# Now we can train the model on the complete dataset
from time import time

t0 = time()
new_ratings_model = ALS.train(modelRDD, rank=5, iterations=5, seed=5)
tt = time() - t0

print("New model trained in {:.2f} seconds".format(round(tt,3)))

New model trained in 81.20 seconds


In [21]:
artist_name_lookup_table = _RDD.map(lambda x: (x[2],x[3])).sortByKey().cache()
artist_id_lookup_table = _RDD.map(lambda x: (x[3],x[2])).sortByKey().cache()

In [46]:
set(artist_id_lookup_table.lookup('satisf*cktion'))

{'3018'}

In [ ]:
set(artist_name_lookup_table.lookup('23199'))
{'alain dzukam'}
{'gian - home studio'}

In [47]:
# Let's test the new model with a few recommendations:
# For the final recommender we should probably scale the recommendations numbers between perhaps 1 and 10. For now I'll
# Regard 1 as does not like, 200 as likes and 500 as loves.

johnsRecommendations = [('56','3018',.0001),
                       ('56','153162',.0001), #Not an actual artist
                       ('56','288963',.0001),# Alain Dzukam. Another movie
                        ('56','14368',.0001), #David guetta. This is a movie
                        ('56','204066',.1), #Rihanna
                        ('56','174344',.0001),# Kaiser Chiefs
                        ('56','95545',.0001), #Iron maiden
                        ('56','58275',.0001),# Dire straits 
                        ('56','43548',.18),#Regina Spektor
                        ('56','68390',.19), #Artst is 'edith piaf'
                        ('56','281461',.20), #Artist is 'maria callas' 
                        ('56','201482',.17),# Atist is 'bryn terfel'
                        ('56','243041',.17), # Anna netrebko
                        ('56','244054',.17), # The residents
                        ('56','256484',.18), #The Legendary Pink Dots
                        ('56','206805',.17), #bjork
                        ('56','127801',.0001), #Madonna
                        ('56','96194',.16), #The killers
                        ('56','216449',.0001), #Taylor Swift
                        ('56','113418',.0001), #Miley Cyrus
                        ('56','253951',.0001), # recommended: Madonna feat. Justin Timberlake. Lets see if model responds
                        ('56','279301',.0001) # recommended: t-pain
                       
                       ]
                       


In [48]:
new_user_ratings_RDD = sc.parallelize(johnsRecommendations)
new_user_ratings_RDD.take(3)

[('56', '3018', 0.0001), ('56', '153162', 0.0001), ('56', '288963', 0.0001)]

In [49]:
newRDD = modelRDD.union(new_user_ratings_RDD)

In [50]:
newRDD.take(5)

[('232513467', '203477', 0.6123032904148784),
 ('660133657', '139067', 0.030042918454935622),
 ('565863613', '93392', 0.07868383404864092),
 ('428504185', '126798', 0.005722460658082976),
 ('666373732', '149036', 0.030042918454935622)]

In [51]:
modelFM = ALS.train(newRDD, rank=5, iterations=5, seed=5)

In [ ]:
model_path = os.path.join('/home/jovyan', 'model', 'lastFMExplicit')

# Save and load model
modelFM.save(sc, model_path)
#same_model = MatrixFactorizationModel.load(sc, model_path

In [52]:
#Get the artists to iterate through
artists = newRDD.map(lambda x: (x[1]))

In [53]:
artists.take(5)

['203477', '139067', '93392', '126798', '149036']

In [54]:
_rated_artists = set(new_user_ratings_RDD.map(lambda x: (x[1])).collect())
unrated_artists = newRDD.filter(lambda x: x[1] not in _rated_artists ).map(lambda x: ('56', x[1]))

In [55]:
new_user_recommendations_RDD = modelFM.predictAll(unrated_artists)

In [56]:
new_user_recommendations_RDD.map(lambda x: (x[2])).max()

0.5211543805135976

In [57]:
top_items = new_user_recommendations_RDD.distinct().takeOrdered(20, key=lambda x: -x[2])


In [58]:
print('Your top ten recommended artists:')
for x in top_items:
    artist = str(x[1])
    print(set(artist_name_lookup_table.lookup(artist)))

Your top ten recommended artists:
{'bippp (v.a.)'}
{'heinz holliger - zehetmair - larcher - holliger - etc.'}
{'dj philip'}
{'active coma'}
{'alberto cortéz y facundo cabral'}
{'jens rachut'}
{'marco antonios solis'}
{'bellhouse'}
{'mike shiflet'}
{'digital mystikz & loefah'}
{'andy vores'}
{'andrew davis'}
{'electric eel-shock'}
{'alexander schatten'}
{'lech jankowski'}
{'manu chao !'}
{'c.a.r.n.e.'}
{'컬투'}
{'die dødelsäcke'}
{'amor-te'}


In [45]:
model_path = os.path.join('/home/spark', 'model', 'lastFMExplicit')

# Save and load model
modelFM.save(sc, model_path)
#same_model = MatrixFactorizationModel.load(sc, model_path